# Algoritmo mediante Pyomo y Pandapower para realizar optimización de potencia reactiva, multiárea y multiperiodo

In [14]:
# librerías necesarias

import pyomo.environ as pyomo
import pandapower as pp
import pandapower.grid_equivalents as ge
import pandapower.networks as pp_net
import numpy as np

In [15]:
# cargamos el modelo eléctrico 
net = pp_net.case39()
net.sn_mva = 1.0
net.load['p_mw'] = 0.75*net.load['p_mw']
net.load['q_mvar'] = 0.75*net.load['q_mvar']
pp.runpp(net)
net

This pandapower network includes the following parameter tables:
   - bus (39 elements)
   - load (21 elements)
   - gen (9 elements)
   - ext_grid (1 element)
   - line (35 elements)
   - trafo (11 elements)
   - poly_cost (10 elements)
   - bus_geodata (39 elements)
 and the following results tables:
   - res_bus (39 elements)
   - res_line (35 elements)
   - res_trafo (11 elements)
   - res_ext_grid (1 element)
   - res_load (21 elements)
   - res_gen (9 elements)

In [16]:
# ajustamos los límites de Taps de los trafos
tab_minimo = 1
tab_neutral = 9
tab_maximo = 21
net.trafo['tap_min'] = net.trafo['tap_min'].replace([np.nan], tab_minimo)
net.trafo['tap_max'] = net.trafo['tap_max'].replace([np.nan], tab_maximo)
net.trafo['tap_neutral'] = net.trafo['tap_neutral'].replace([0], tab_neutral)

In [17]:
# variables iniciales 
pp.runpp(net)
ten_init = np.array(net.res_bus['vm_pu'])
mvar_base = np.array(net.res_gen['q_mvar'])
mvar_max = np.array(net.gen['max_q_mvar'])
cant_gen = net.gen.shape[0]
cant_trafo = net.trafo.shape[0]

In [18]:
# creamos el modelo de optimización
model = pyomo.ConcreteModel();

In [19]:
# creamos las variables del modelo, Tap de trafos
for i in range(cant_trafo):
    model.add_component('tab_trafo_%s'%i, pyomo.Var(
        within=pyomo.PositiveIntegers, 
        bounds=(net.trafo.at[i, 'tap_min'], net.trafo.at[i, 'tap_max']), 
        initialize=net.trafo.at[i, 'tap_pos']
    ))

In [20]:
# creamos las variables del modelo, setpoint generación
for i in range(cant_gen):
    model.add_component('setpoint_gen_%s'%i, pyomo.Var(
        within=pyomo.PositiveReals, 
        bounds=(0.8, 1.2), 
        initialize=net.gen.at[i, 'vm_pu']
    ))

In [25]:
#Objective function definition
def objetive(model):
    pp.runpp(net)
    #tap_trafos = [model.find_component('tab_trafo_%s'%i)
    #                        for i in range(cant_trafo)]
    net.trafo['tap_pos'] = [model.-find_component('tab_trafo_%s'%i)
                            for i in range(cant_trafo)]
    #setpoint_gen = [model.find_component('setpoint_gen_%s'%i)
    #                    for i in range(cant_gen)]
    net.gen['vm_pu'] = [model.find_component('setpoint_gen_%s'%i)
                        for i in range(cant_gen)]
    change_ten = np.sum(
        np.square(ten_init - np.array(net.res_bus['vm_pu']))
    ) 
    change_mvar = np.sum(
        np.square(np.array(net.res_gen['q_mvar']) / mvar_base)
    )
    print('Result= %s'%(change_ten + change_mvar))
    return change_ten + change_mvar
model.obj = pyomo.Objective(expr=objetive, sense = pyomo.minimize)

    'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
Result= 9.0


In [26]:
#Constraint definition
def rule1(model):
    result = np.sum(np.array([model.find_component('tab_trafo_%s'%i)
                            for i in range(cant_trafo)]))
    return result<=200
model.eq1 = pyomo.Constraint(rule = rule1, doc = 'Constraint 1')

    'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [27]:
#Solver options
results = pyomo.SolverFactory('solver/bonmin').solve(model);

In [ ]:
results.write()
print("\n RESULTS \n");

print('Cost of advertisement campaign = ', model.obj())
for i in range(cant_trafo):
    print('trafo_%s tap = '%i, model.find_component('tab_trafo_%s'%i).value)
for i in range(cant_gen):
    print('gen_%s setpint = '%i, model.find_component('setpoint_gen_%s'%i).value)


In [45]:
import pandapower as pp
import pandapower.networks as nw
import pyomo.environ as pyo

# cargar una red eléctrica de ejemplo de pandapower
net = nw.case9()
pp.runpp(net)

# crear un modelo de optimización con Pyomo
model = pyo.ConcreteModel()

# agregar variables de decisión para controlar la potencia reactiva de cada generador
model.q = pyo.Var(net.gen['vm_pu'], within=pyo.Reals)

# agregar una función objetivo para minimizar la diferencia entre el set point y la potencia reactiva entregada por cada generador
model.objective = pyo.Objective(
    expr=sum((model.q[i] - net.res_gen.loc[i, "vm_pu"])**2 for i in net.gen['vm_pu']),
    sense=pyo.minimize
)

# agregar restricciones de demanda de potencia y capacidad de la red
for i in net.bus.index:
    # model.bus_power_demand = pyo.Constraint(
    #     expr=sum(
    #         net.gen.loc[j, "s_nom"] * model.q[j]
    #         for j in net.gen[net.gen.bus == i].index
    #     )
    #     >= net.bus.loc[i, "pd"]
    # )
    model.add_component("contrains_%s"%i,
        pyo.Constraint(
            expr=sum(
                net.gen.loc[j, "p_mw"] * model.q[j]
                for j in net.gen[net.gen.bus == i].index
            )
            <= net.res_bus.loc[i, "vm_pu"] * net.bus.loc[i, "max_vm_pu"]
        )
    )

# resolver el modelo de optimización con un solucionador de Pyomo
solver = pyo.SolverFactory("solver/bonmin")
results = solver.solve(model)

# imprimir los resultados
print(f"Solución óptima: {model.objective()}")
print("Potencia reactiva entregada por cada generador:")
for i in net.gen.index:
    print(f"Generador {i}: {model.q[i].value} MVAr")



ERROR: Rule failed when generating expression for Constraint contrains_0 with
    index None: ValueError: Constraint 'contrains_0' does not have a proper
    value. Found 'True' Expecting a tuple or relational expression. Examples:
       sum(model.costs) == model.income (0, model.price[item], 50)
ERROR: Constructing component 'contrains_0' from data=None failed: ValueError:
    Constraint 'contrains_0' does not have a proper value. Found 'True'
    Expecting a tuple or relational expression. Examples:
       sum(model.costs) == model.income (0, model.price[item], 50)


ValueError: Constraint 'contrains_0' does not have a proper value. Found 'True'
Expecting a tuple or relational expression. Examples:
   sum(model.costs) == model.income
   (0, model.price[item], 50)

In [42]:
model.q

In [38]:
net.bus

,in_service,max_vm_pu,min_vm_pu,name,type,vn_kv,zone
0,True,1.1,0.9,1,b,345.0,1.0
1,True,1.1,0.9,2,b,345.0,1.0
2,True,1.1,0.9,3,b,345.0,1.0
3,True,1.1,0.9,4,b,345.0,1.0
4,True,1.1,0.9,5,b,345.0,1.0
5,True,1.1,0.9,6,b,345.0,1.0
6,True,1.1,0.9,7,b,345.0,1.0
7,True,1.1,0.9,8,b,345.0,1.0
8,True,1.1,0.9,9,b,345.0,1.0


In [41]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-71.954702,-24.068958
1,1.000000,9.668741,-163.000000,-14.460120
2,1.000000,4.771073,-85.000000,3.649026
3,0.987007,-2.406644,0.000000,0.000000
4,0.975472,-4.017264,90.000000,30.000000
5,1.003375,1.925602,0.000000,0.000000
6,0.985645,0.621545,100.000000,35.000000
7,0.996185,3.799120,0.000000,0.000000
8,0.957621,-4.349934,125.000000,50.000000
